In [82]:
import pandas as pd
import os
import numpy as np

In [83]:
df = pd.read_csv('/Users/marikhomeriki/code/marikhomeriki/raw_data/train.csv', header=None)

In [84]:
df_test = pd.read_csv('/Users/marikhomeriki/code/marikhomeriki/raw_data/test.csv', header=None)

In [85]:
df_test = df_test.rename({0: 'label', 1: 'text'}, axis = 1)

In [86]:
df = df.rename({0: 'label', 1: 'text'}, axis = 1)

In [87]:
df_test

,label,text
0,2,"Contrary to other reviews, I have zero complai..."
1,1,Last summer I had an appointment to get new ti...
2,2,"Friendly staff, same starbucks fair you get an..."
3,1,The food is good. Unfortunately the service is...
4,2,Even when we didn't have a car Filene's Baseme...
...,...,...
37995,1,If I could give 0...I would. Don't do it.
37996,2,Items Selected:\nChocolate Cinnamon Horn\nSmal...
37997,1,Expensive lunch meals. Fried pickles were goo...
37998,1,Highly overpriced and food was cold. Our waitr...


In [88]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(data, percentage_of_sentences=None):
    
    train_sentences = data['text']
    y_train = data['label']
    
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
    X_train = [text_to_word_sequence(_) for _ in train_sentences]
    
  
    return X_train, y_train



In [89]:
X_train, y_train = load_data(df, percentage_of_sentences=10)

In [90]:
X_test, y_test = load_data(df_test, percentage_of_sentences=10)

In [53]:
type(X_train)

list

In [16]:
len_train = int(10/100*len(df_test['text']))
X_test = df_test['text']
X_test = X_test[:len_train]

In [92]:
y_test_adj = y_test -1

In [93]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
    
# We apply the tokenization to the train and test set
X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)


X_train_pad = pad_sequences(X_train_token, maxlen=50, dtype='float32', padding="post")
X_test_pad = pad_sequences(X_test_token, maxlen=50, dtype='float32', padding="post")

vocab_size = len(tokenizer.word_index)

In [136]:
import pickle

In [137]:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [94]:
type(X_train_pad)

numpy.ndarray

In [95]:
from tensorflow.keras import Sequential, layers

def init_cnn_model(vocab_size):
    model = Sequential()
    model.add(layers.Embedding(input_dim=vocab_size + 1, output_dim=20, mask_zero=True, input_length=50))
    model.add(layers.Conv1D(32, 3))
    model.add(layers.Flatten())
    model.add(layers.Dense(50, activation="relu"))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model_cnn = init_cnn_model(vocab_size)

In [96]:
model_cnn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 20)            1378040   
                                                                 
 conv1d_2 (Conv1D)           (None, 48, 32)            1952      
                                                                 
 flatten_2 (Flatten)         (None, 1536)              0         
                                                                 
 dense_4 (Dense)             (None, 50)                76850     
                                                                 
 dense_5 (Dense)             (None, 1)                 51        
                                                                 
Total params: 1,456,893
Trainable params: 1,456,893
Non-trainable params: 0
_________________________________________________________________


In [97]:
y_train_adj = y_train -1

In [98]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model_cnn.fit(X_train_pad, y_train_adj, 
          epochs=20, 
          batch_size=32,
          validation_split=0.3,
          callbacks=[es]
         )


Epoch 1/20
1225/1225 [==============================] - 7s 5ms/step - loss: 0.3593 - accuracy: 0.8326 - val_loss: 0.2841 - val_accuracy: 0.8824
Epoch 2/20
1225/1225 [==============================] - 6s 5ms/step - loss: 0.1914 - accuracy: 0.9231 - val_loss: 0.3276 - val_accuracy: 0.8679
Epoch 3/20
1225/1225 [==============================] - 6s 5ms/step - loss: 0.0718 - accuracy: 0.9728 - val_loss: 0.4842 - val_accuracy: 0.8552
Epoch 4/20
1225/1225 [==============================] - 6s 5ms/step - loss: 0.0235 - accuracy: 0.9922 - val_loss: 0.7559 - val_accuracy: 0.8468
Epoch 5/20
1225/1225 [==============================] - 6s 5ms/step - loss: 0.0121 - accuracy: 0.9956 - val_loss: 0.9035 - val_accuracy: 0.8511
Epoch 6/20
1225/1225 [==============================] - 6s 5ms/step - loss: 0.0131 - accuracy: 0.9956 - val_loss: 0.9769 - val_accuracy: 0.8510


In [99]:
res = model_cnn.evaluate(X_test_pad, y_test_adj)

119/119 [==============================] - 0s 649us/step - loss: 0.2800 - accuracy: 0.8837


In [100]:
model_cnn

In [101]:
df_test = pd.read_csv('/Users/marikhomeriki/code/marikhomeriki/raw_data/data_dishoom.csv', header = None)
df_test = df_test.rename({1: 'text'}, axis = 1)
df_test = df_test.dropna()
df_test = pd.DataFrame(df_test.text)

In [103]:
df_test = [text_to_word_sequence(_) for _ in df_test.text]

In [105]:
df_test_token = tokenizer.texts_to_sequences(df_test)

df_token_pad = pad_sequences(df_test_token, maxlen=50, dtype='float32', padding="post")


In [126]:
df_pred = model_cnn.predict(df_token_pad)

16/16 [==============================] - 0s 8ms/step


In [127]:
df_pred = [0 if pred <0.5 else 1 for pred in df_pred]

In [129]:
df_pred = ['Negative' if sent == 0 else 'Positive' for sent in df_pred]


In [130]:
from collections import Counter


In [132]:
Counter(df_pred)

Counter({'Negative': 72, 'Positive': 428})

In [36]:
y_pred = model_cnn.predict(X_test_pad)

119/119 [==============================] - 0s 745us/step


In [37]:
y_pred

array([[0.19294204],
       [0.03920677],
       [0.9462042 ],
       ...,
       [0.04279505],
       [0.08128761],
       [0.25588748]], dtype=float32)

In [39]:
model_cnn.save("CNN_model")

INFO:tensorflow:Assets written to: CNN_model/assets


INFO:tensorflow:Assets written to: CNN_model/assets


In [38]:
y_pred = np.round(y_pred)

In [61]:
y_pred = y_pred+1

In [81]:
y_pred
y_sent = ['Negative' if sent == 1 else 'Positive' for sent in y_pred]



In [73]:
y_sent = pd.DataFrame(y_sent)

In [78]:
result = y_sent[0].value_counts(normalize=True)

In [80]:
result

Negative    0.561842
Positive    0.438158
Name: 0, dtype: float64

In [49]:
X_test = pd.DataFrame(X_test)

In [50]:
X_test['label_pred'] = y_pred

In [51]:
X_test

,text,label_pred
0,"Contrary to other reviews, I have zero complai...",1.0
1,Last summer I had an appointment to get new ti...,1.0
2,"Friendly staff, same starbucks fair you get an...",2.0
3,The food is good. Unfortunately the service is...,1.0
4,Even when we didn't have a car Filene's Baseme...,2.0
...,...,...
3795,Came here on a Tues night at 6pm. Warning. If ...,1.0
3796,I've been here since my teeny bopper years and...,2.0
3797,Friendly staff and quick service for auto tint...,1.0
3798,Please be warned! Do not patronize Home Fabric...,1.0


In [52]:
X_test['sentiment'] = X_test['label_pred'].apply(lambda x: 'negative' if x == 1.0 else 'positive')

In [53]:
X_test

,text,label_pred,sentiment
0,"Contrary to other reviews, I have zero complai...",1.0,negative
1,Last summer I had an appointment to get new ti...,1.0,negative
2,"Friendly staff, same starbucks fair you get an...",2.0,positive
3,The food is good. Unfortunately the service is...,1.0,negative
4,Even when we didn't have a car Filene's Baseme...,2.0,positive
...,...,...,...
3795,Came here on a Tues night at 6pm. Warning. If ...,1.0,negative
3796,I've been here since my teeny bopper years and...,2.0,positive
3797,Friendly staff and quick service for auto tint...,1.0,negative
3798,Please be warned! Do not patronize Home Fabric...,1.0,negative


In [54]:
X_test.drop(columns='label_pred')

,text,sentiment
0,"Contrary to other reviews, I have zero complai...",negative
1,Last summer I had an appointment to get new ti...,negative
2,"Friendly staff, same starbucks fair you get an...",positive
3,The food is good. Unfortunately the service is...,negative
4,Even when we didn't have a car Filene's Baseme...,positive
...,...,...
3795,Came here on a Tues night at 6pm. Warning. If ...,negative
3796,I've been here since my teeny bopper years and...,positive
3797,Friendly staff and quick service for auto tint...,negative
3798,Please be warned! Do not patronize Home Fabric...,negative


In [66]:
X_test.sentiment.value_counts()

AttributeError: 'list' object has no attribute 'sentiment'